In [1]:
from tqdm import trange
from flygym.envs.nmf_mujoco import MuJoCoParameters
from flygym.arena.mujoco_arena import OdorArena
from flygym.state import stretched_pose
from flygym.util.config import all_leg_dofs

from cpg_controller import NMFCPG

In [2]:
odor_source = [[2, -1, 1.5], [5, 1, 1.5]]
peak_intensity = [[80, 0], [0, 100]]

arena = OdorArena(
    size=(100, 100),
    odor_source=odor_source,
    peak_intensity=peak_intensity,
    diffuse_func=lambda x: x ** -2,
)

sim_params = MuJoCoParameters(
    timestep=1e-4, render_mode="saved", render_playspeed=0.1, enable_olfaction=True
)

nmf_cpg = NMFCPG(
    sim_params=sim_params,
    arena=arena,
    init_pose=stretched_pose,
    actuated_joints=all_leg_dofs,
)

In [3]:
obs, _ = nmf_cpg.reset()
obs_list = [obs]

actions = [[0, 0], [1, 1], [-1, -1], [-1, 1], [1, -1]]
time_per_action = 0.5
steps_per_action = int(time_per_action / sim_params.timestep)

for action in actions:
    for i in trange(steps_per_action, desc=f"Drive signal: {action}"):
        obs, _, _, _, _ = nmf_cpg.step(action)
        obs_list.append(obs)
        nmf_cpg.render()

nmf_cpg.nmf.save_video("outputs/turning_test.mp4")

Drive signal: [1, -1]: 100%|██████████| 5000/5000 [00:13<00:00, 381.09it/s]
